In [ ]:
import chess.engine

# Définir le chemin vers l'exécutable de Stockfish
STOCKFISH_PATH = "/usr/games/stockfish" 

# Initialiser le moteur Stockfish
engine = chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH)


In [ ]:
def evaluer_coup(echiquier, coup):
    """Analyse un coup avec Stockfish et retourne une évaluation qualitative."""

    # Évaluation avant le coup
    info_avant = engine.analyse(echiquier, chess.engine.Limit(depth=18))
    eval_avant = info_avant["score"].relative.score(mate_score=10000)  # Centipions
    
    # Jouer le coup
    echiquier.push(coup)

    # Évaluation après le coup
    info_apres = engine.analyse(echiquier, chess.engine.Limit(depth=18))
    eval_apres = info_apres["score"].relative.score(mate_score=10000)  # Centipions

    # Annuler le coup pour ne pas modifier l'échiquier
    echiquier.pop()

    # Différence d'évaluation
    variation = eval_apres - eval_avant

    # Déterminer la classification du coup
    if variation >= 300:
        return "!!"  # Coup brillant
    elif 100 <= variation < 300:
        return "!"
    elif -50 <= variation < 0:
        return "?!?"  # Imprécision
    elif -100 <= variation < -50:
        return "?"  # Erreur
    elif variation < -100:
        return "??"  # Grosse erreur
    else:
        return ""  # Coup normal


In [ ]:
from stockfish import Stockfish

stockfish = Stockfish("/usr/games/stockfish")  # Mets le chemin correct ici
stockfish.set_skill_level(10)

print(stockfish.get_best_move())  # Vérifie si Stockfish fonctionne


In [23]:
import ipycanvas
import ipywidgets as widgets
import os
import chess
import chess.pgn
from stockfish import Stockfish

taille_case = 50
echiquier_size = 8
chemin_images = "chess_images"
canvas = ipycanvas.Canvas(width=taille_case * echiquier_size, height=taille_case * echiquier_size)
display(canvas)

stockfish = Stockfish("/usr/games/stockfish") 
stockfish.set_skill_level(10)

echiquier = chess.Board()

def charger_partie_pgn(fichier_pgn):
    with open(fichier_pgn, "r") as pgn:
        partie = chess.pgn.read_game(pgn)
    coups = []
    for node in partie.mainline():
        coups.append(node.move.uci())  # Convertir chaque coup en UCI
    return coups

def dessiner_echiquier():
    for y in range(echiquier_size):
        for x in range(echiquier_size):
            color = "white" if (x + y) % 2 == 0 else "grey"
            canvas.fill_style = color
            canvas.fill_rect(x * taille_case, y * taille_case, taille_case, taille_case)

def dessiner_piece_image(image_path, x, y):
    if os.path.exists(image_path):
        with open(image_path, "rb") as file:
            image = widgets.Image(value=file.read(), format='png')
        canvas.draw_image(image, x * taille_case, y * taille_case, taille_case, taille_case)

def mettre_a_jour_echiquier():
    canvas.clear()
    dessiner_echiquier()
    for case in chess.SQUARES:
        piece = echiquier.piece_at(case)
        if piece:
            x, y = chess.square_file(case), 7 - chess.square_rank(case)
            image_path = os.path.join(chemin_images, f"{'w' if piece.color else 'b'}{piece.symbol().upper()}.png")
            dessiner_piece_image(image_path, x, y)

def appliquer_coup(coup):
    global echiquier
    try:
        # Convertir la chaîne de caractère en objet Move
        move = chess.Move.from_uci(coup)  
        if move in echiquier.legal_moves:
            echiquier.push(move)  # Appliquer le coup  

            eval_avant = stockfish.get_evaluation().get("value", 0)  # Récupération de l'évaluation avant le coup
            mettre_a_jour_echiquier()

            annotation = evaluer_coup(echiquier, move)
            eval_apres = stockfish.get_evaluation().get("value", 0)  # Évaluation après le coup

            print("Appel de afficher_annotation_image()")
            afficher_annotation_image(annotation, move, eval_avant, eval_apres)
        else:
            print(f"Coup {coup} illégal sur l'échiquier.")

    except Exception as e:
        print(f"Erreur lors de l'application du coup {coup}: {e}")


def evaluer_coup(echiquier, coup):
    """Analyse un coup avec Stockfish et retourne une évaluation qualitative."""
    fen_avant = echiquier.fen()  
    stockfish.set_fen_position(fen_avant) 
    info_avant = stockfish.get_evaluation()

    if info_avant is None or 'value' not in info_avant:
        print("Erreur: Stockfish n'a pas retourné de valeur pour l'évaluation avant le coup.")
        return "Erreur d'évaluation"

    eval_avant = info_avant['value']

    echiquier.push(coup)  # Appliquer le coup

    fen_apres = echiquier.fen()  
    stockfish.set_fen_position(fen_apres)
    info_apres = stockfish.get_evaluation()

    if info_apres is None or 'value' not in info_apres:
        print("Erreur: Stockfish n'a pas retourné de valeur pour l'évaluation après le coup.")
        echiquier.pop()  
        return "Erreur d'évaluation"

    eval_apres = info_apres['value']  

    echiquier.pop()  

    variation = eval_apres - eval_avant

    if variation >= 300:
        annotation = "!!"  
    elif 100 <= variation < 300:
        annotation = "!"  
    elif -50 <= variation < 0:
        annotation = "?!?"  
    elif -100 <= variation < -50:
        annotation = "?" 
    elif variation < -100:
        annotation = "??" 
    else:
        annotation = ""  

    return annotation

def afficher_annotation_image(annotation, move, eval_avant, eval_apres):
    # Cette fonction peut afficher l'annotation sur l'échiquier
    print(f"Annotation pour le coup {move}: {annotation}")
    print(f"Évaluation avant: {eval_avant}, Évaluation après: {eval_apres}")

def afficher_evaluation(b):
    if indice_coup > 0:
        dernier_coup = coups[indice_coup - 1]  # Dernier coup joué
        annotation = evaluer_coup(echiquier, dernier_coup)
        case = dernier_coup.to_square()  # Pour obtenir la case à partir du coup

        x, y = chess.square_file(case), 7 - chess.square_rank(case)
        annotation_x = x * taille_case + taille_case // 4  
        annotation_y = y * taille_case + taille_case // 2  

        canvas.fill_style = "blue"
        canvas.font = "bold 15px Arial"
        canvas.fill_text(annotation, annotation_x, annotation_y)
    else:
        print("Aucun coup joué, impossible d'afficher l'évaluation.")

def rejouer_partie(fichier_pgn):
    global echiquier, coups, indice_coup
    echiquier = chess.Board()
    coups = charger_partie_pgn(fichier_pgn)
    indice_coup = 0
    mettre_a_jour_echiquier()

    bouton_suivant = widgets.Button(description="Coup suivant")
    bouton_suivant.on_click(jouer_prochain_coup)
    bouton_precedent = widgets.Button(description="Coup precedent")
    bouton_precedent.on_click(jouer_ancien_coup)
    display(bouton_suivant)
    display(bouton_precedent)
    bouton_eval = widgets.Button(description="Afficher Évaluation")
    bouton_eval.on_click(afficher_evaluation)
    display(bouton_eval)

def jouer_prochain_coup(b):
    global indice_coup
    if indice_coup < len(coups):
        appliquer_coup(coups[indice_coup])
        indice_coup += 1

def jouer_ancien_coup(b):
    global indice_coup
    if indice_coup > 0:
        echiquier.pop()  
        indice_coup -= 1
        mettre_a_jour_echiquier()

# Rejouer une partie (remplacer par ton propre fichier PGN)
rejouer_partie('Loowolf_vs_TelophileGOAT_2025.02.19.pgn')


Canvas(height=400, width=400)

Button(description='Coup suivant', style=ButtonStyle())

Button(description='Coup precedent', style=ButtonStyle())

Button(description='Afficher Évaluation', style=ButtonStyle())

In [24]:
import ipycanvas
import ipywidgets as widgets
import os
import chess
import chess.pgn
from stockfish import Stockfish  # Assurez-vous que Stockfish est installé et fonctionnel

# Dimensions de l'échiquier
taille_case = 50
echiquier_size = 8
chemin_images = "chess_images"
canvas = ipycanvas.Canvas(width=taille_case * echiquier_size, height=taille_case * echiquier_size)
display(canvas)

# Créer un moteur Stockfish
stockfish = Stockfish("/usr/games/stockfish")  # Mettre le chemin d'accès correct pour Stockfish
stockfish.set_skill_level(10)

# Créer l'échiquier de départ
echiquier = chess.Board()

# Charger une partie PGN
def charger_partie_pgn(fichier_pgn):
    with open(fichier_pgn, "r") as pgn:
        partie = chess.pgn.read_game(pgn)
    return list(partie.mainline_moves())

# Dessiner l'échiquier sur le canvas
def dessiner_echiquier():
    for y in range(echiquier_size):
        for x in range(echiquier_size):
            color = "white" if (x + y) % 2 == 0 else "grey"
            canvas.fill_style = color
            canvas.fill_rect(x * taille_case, y * taille_case, taille_case, taille_case)

# Dessiner les pièces sur l'échiquier
def dessiner_piece_image(image_path, x, y):
    if os.path.exists(image_path):
        with open(image_path, "rb") as file:
            image = widgets.Image(value=file.read(), format='png')
        canvas.draw_image(image, x * taille_case, y * taille_case, taille_case, taille_case)

# Mettre à jour l'échiquier
def mettre_a_jour_echiquier():
    canvas.clear()
    dessiner_echiquier()
    for case in chess.SQUARES:
        piece = echiquier.piece_at(case)
        if piece:
            x, y = chess.square_file(case), 7 - chess.square_rank(case)
            image_path = os.path.join(chemin_images, f"{'w' if piece.color else 'b'}{piece.symbol().upper()}.png")
            dessiner_piece_image(image_path, x, y)


# Appliquer un coup et évaluer
def appliquer_coup(coup):
    global echiquier
    try:
        # Convertir le coup en notation UCI
        move = chess.Move.from_uci(str(coup))

        # Vérifier que le coup est valide
        if move not in echiquier.legal_moves:
            print(f"Le coup {coup} n'est pas légal.")
            return
        
        # Appliquer le coup
        echiquier.push(move)  
        mettre_a_jour_echiquier()  # Mettre à jour l'affichage
        
        # Évaluer le coup avec Stockfish
        annotation = evaluer_coup(echiquier, move)
        afficher_annotation(annotation)  # Afficher l'annotation

    except Exception as e:
        print(f"Erreur lors de l'application du coup {coup}: {e}")


# Fonction pour évaluer un coup avec Stockfish
def evaluer_coup(echiquier, coup):
    """Analyse un coup avec Stockfish et retourne une évaluation qualitative."""

    # Évaluation avant le coup
    fen_avant = echiquier.fen()  # Récupérer la position avant le coup
    stockfish.set_fen_position(fen_avant)  # Envoyer la position à Stockfish
    info_avant = stockfish.get_evaluation()  # Obtenir l'évaluation avant le coup
    eval_avant = info_avant['value']  # Récupérer l'évaluation (en centipions)
    
    # Jouer le coup
    echiquier.push(coup)

    # Évaluation après le coup
    fen_apres = echiquier.fen()  # Récupérer la position après le coup
    stockfish.set_fen_position(fen_apres)  # Envoyer la position à Stockfish
    info_apres = stockfish.get_evaluation()  # Obtenir l'évaluation après le coup
    eval_apres = info_apres['value']  # Récupérer l'évaluation (en centipions)

    # Annuler le coup pour ne pas modifier l'échiquier
    echiquier.pop()

    # Différence d'évaluation
    variation = eval_apres - eval_avant

    # Déterminer la classification du coup
    if variation >= 300:
        return "!!"  # Coup brillant
    elif 100 <= variation < 300:
        return "!"
    elif -50 <= variation < 0:
        return "?!?"  # Imprécision
    elif -100 <= variation < -50:
        return "?"  # Erreur
    elif variation < -100:
        return "??"  # Grosse erreur
    else:
        return ""  # Coup normal

# Fonction pour afficher l'annotation sur le canvas
def afficher_annotation(annotation):
    canvas.fill_style = "black"
    canvas.font = "20px Arial"
    canvas.fill_text(f"Annotation: {annotation}", 10, 20)  # Affiche l'annotation en haut à gauche

# Charger une partie et la rejouer
def rejouer_partie(fichier_pgn):
    global echiquier, coups, indice_coup
    echiquier = chess.Board()
    coups = charger_partie_pgn(fichier_pgn)
    indice_coup = 0
    mettre_a_jour_echiquier()

    # Boutons pour avancer et reculer dans les coups
    bouton_suivant = widgets.Button(description="Coup suivant")
    bouton_suivant.on_click(jouer_prochain_coup)
    bouton_precedent = widgets.Button(description="Coup precedent")
    bouton_precedent.on_click(jouer_ancien_coup)
    display(bouton_suivant)
    display(bouton_precedent)

# Jouer le prochain coup
def jouer_prochain_coup(b):
    global indice_coup
    if indice_coup < len(coups):
        appliquer_coup(coups[indice_coup])
        indice_coup += 1

# Jouer l'ancien coup
def jouer_ancien_coup(b):
    global indice_coup
    if indice_coup > 0:
        echiquier.pop()  # Annuler le dernier coup
        indice_coup -= 1
        mettre_a_jour_echiquier()

# Lancer la partie (changez le nom du fichier PGN si nécessaire)
rejouer_partie('Loowolf_vs_TelophileGOAT_2025.02.19.pgn')


Canvas(height=400, width=400)

Button(description='Coup suivant', style=ButtonStyle())

Button(description='Coup precedent', style=ButtonStyle())

In [ ]:
def appliquer_coup(coup):
    global echiquier
    move_chess = None
    move = move_chess 
    move = chess.Move.from_uci(str(coup))  # Convertir le coup en notation UCI
    if move in echiquier.legal_moves:  # Vérifie si le coup est légal
        echiquier.push(move)  # Appliquer le coup
        mettre_a_jour_echiquier()  # Mettre à jour l'affichage
        annotation = evaluer_coup(echiquier, move_chess )  # Évaluer le coup
        afficher_annotation(annotation)  # Afficher l'annotation
    else:
        print(f"Le coup {coup} n'est pas valide ou ne peut pas être joué à ce moment.")

In [32]:
import ipycanvas
import ipywidgets as widgets
import os
import chess
import chess.pgn
from stockfish import Stockfish  


taille_case = 50
echiquier_size = 8
chemin_images = "chess_images"
canvas = ipycanvas.Canvas(width=taille_case * echiquier_size, height=taille_case * echiquier_size)
display(canvas)


stockfish = Stockfish("/usr/games/stockfish") 
stockfish.set_skill_level(10)


echiquier = chess.Board()


def charger_partie_pgn(fichier_pgn):
    with open(fichier_pgn, "r") as pgn:
        partie = chess.pgn.read_game(pgn)
    coups = []
    for node in partie.mainline():
        coups.append(node.move)
    return coups

def convertir_coup_en_uci(coup_pgn):
    return coup_pgn.uci()


def dessiner_echiquier():
    for y in range(echiquier_size):
        for x in range(echiquier_size):
            color = "white" if (x + y) % 2 == 0 else "grey"
            canvas.fill_style = color
            canvas.fill_rect(x * taille_case, y * taille_case, taille_case, taille_case)


def dessiner_piece_image(image_path, x, y):
    if os.path.exists(image_path):
        with open(image_path, "rb") as file:
            image = widgets.Image(value=file.read(), format='png')
        canvas.draw_image(image, x * taille_case, y * taille_case, taille_case, taille_case)


def mettre_a_jour_echiquier():
    canvas.clear()
    dessiner_echiquier()
    for case in chess.SQUARES:
        piece = echiquier.piece_at(case)
        if piece:
            x, y = chess.square_file(case), 7 - chess.square_rank(case)
            image_path = os.path.join(chemin_images, f"{'w' if piece.color else 'b'}{piece.symbol().upper()}.png")
            dessiner_piece_image(image_path, x, y)



def appliquer_coup(coup):
    global echiquier
    try:
        move = chess.Move.from_uci(str(coup))  # Convertir correctement le coup UCI

        if move not in echiquier.legal_moves:
            print(f"Le coup {coup} n'est pas légal.")  # Vérifier si le coup est valide
            return

        eval_avant = stockfish.get_evaluation().get("value", 0)  

        echiquier.push(move)  
        mettre_a_jour_echiquier()  

        annotation = evaluer_coup(echiquier, move)  # Passer le bon argument

        eval_apres = stockfish.get_evaluation().get("value", 0)  

        print("Appel de afficher_annotation_image()")
        afficher_annotation_image(annotation, move, eval_avant, eval_apres)  

    except Exception as e:
        print(f"Erreur lors de l'application du coup {coup}: {e}")





def evaluer_coup(echiquier, coup):
    """Analyse un coup avec Stockfish et retourne une évaluation qualitative."""
    try:
        fen_avant = echiquier.fen()  
        print(f"FEN avant le coup: {fen_avant}")
        stockfish.set_fen_position(fen_avant) 
        info_avant = stockfish.get_evaluation()  
        
        if info_avant is None:
            print(f"Erreur: Stockfish n'a pas retourné de valeur pour l'évaluation avant le coup.")
            return "Erreur d'évaluation"
        elif 'value' not in info_avant:
            print(f"Erreur: Stockfish n'a pas retourné un champ 'value' dans l'évaluation avant le coup.")
            return "Erreur d'évaluation"

        eval_avant = info_avant['value']

        echiquier.push(coup)  

        fen_apres = echiquier.fen()  
        print(f"FEN après le coup: {fen_apres}")
        stockfish.set_fen_position(fen_apres)  
        info_apres = stockfish.get_evaluation()  

        if info_apres is None:
            print(f"Erreur: Stockfish n'a pas retourné de valeur pour l'évaluation après le coup.")
            echiquier.pop()  
            return "Erreur d'évaluation"
        elif 'value' not in info_apres:
            print(f"Erreur: Stockfish n'a pas retourné un champ 'value' dans l'évaluation après le coup.")
            echiquier.pop()  
            return "Erreur d'évaluation"

        eval_apres = info_apres['value']  

        echiquier.pop()  

        variation = eval_apres - eval_avant

        if variation >= 300:
            annotation = "!!"  
        elif 100 <= variation < 300:
            annotation = "!"  
        elif -50 <= variation < 0:
            annotation = "?!?"  
        elif -100 <= variation < -50:
            annotation = "?" 
        elif variation < -100:
            annotation = "??" 
        else:
            annotation = ""  

        return annotation

    except Exception as e:
        print(f"Erreur lors de l'évaluation du coup {coup}: {e}")
        return "Erreur d'évaluation"






def afficher_evaluation(b):
    if indice_coup > 0:
        dernier_coup = coups[indice_coup - 1]  # Dernier coup joué
        annotation = evaluer_coup(echiquier, dernier_coup)
        
        
        case = dernier_coup.to_square  

        x, y = chess.square_file(case), 7 - chess.square_rank(case)  
        annotation_x = x * taille_case + taille_case // 4  
        annotation_y = y * taille_case + taille_case // 2  

        canvas.fill_style = "blue"
        canvas.font = "bold 15px Arial"
        canvas.fill_text(annotation, annotation_x, annotation_y)
    else:
        print("Aucun coup joué, impossible d'afficher l'évaluation.")











def rejouer_partie(fichier_pgn):
    global echiquier, coups, indice_coup  
    echiquier = chess.Board()
    coups = charger_partie_pgn(fichier_pgn)
    indice_coup = 0  
    mettre_a_jour_echiquier()

    bouton_suivant = widgets.Button(description="Coup suivant")
    bouton_suivant.on_click(jouer_prochain_coup)

    bouton_precedent = widgets.Button(description="Coup précédent")
    bouton_precedent.on_click(jouer_ancien_coup)

    bouton_eval = widgets.Button(description="Afficher Évaluation")
    bouton_eval.on_click(afficher_evaluation)

    display(bouton_suivant, bouton_precedent, bouton_eval)



def jouer_prochain_coup(b):
    global indice_coup
    if indice_coup < len(coups):
        appliquer_coup(coups[indice_coup])
        indice_coup += 1


def jouer_ancien_coup(b):
    global indice_coup
    if indice_coup > 0:
        echiquier.pop()  
        indice_coup -= 1
        mettre_a_jour_echiquier()




rejouer_partie('Loowolf_vs_TelophileGOAT_2025.02.19.pgn')


Canvas(height=400, width=400)

Button(description='Coup suivant', style=ButtonStyle())

Button(description='Coup précédent', style=ButtonStyle())

Button(description='Afficher Évaluation', style=ButtonStyle())

In [28]:
coups = charger_partie_pgn('Loowolf_vs_TelophileGOAT_2025.02.19.pgn')

print(coup)

h4e4


In [20]:
coups = charger_partie_pgn('Loowolf_vs_TelophileGOAT_2025.02.19.pgn')
for coup in coups:
    print(coup)


g2g3
e7e5
f2f4
d7d6
g1f3
b8c6
d2d4
e5f4
d4d5
c6e5
g3f4
e5f3
e2f3
g8f6
f1h3
c8h3
b1c3
d8e7
c3e4
e8c8
c2c3
g7g6
c1e3
f6e4
f3e4
e7e4
d1d2
e4h1
e1e2
h1a1
d2d4
a1f1
e2d2
f1g2
d2d3
g2h2
d4a7
c7c6
d5c6
b7c6
a7a6
c8d7
a6b7
d7e8
b7c6
d8d7
c6c8
e8e7
f4f5
h3f5
d3d4
h2h4
d4d5
h4e4


In [17]:
stockfish.set_position(['e2e4'])  # Position initiale ou un coup quelconque
info = stockfish.get_evaluation()
print(info)  # Vérifie l'output pour s'assurer que l'évaluation fonctionne


{'type': 'cp', 'value': 38}


In [1]:
import ipycanvas
import ipywidgets as widgets
import os
import chess
import chess.pgn
from stockfish import Stockfish  

# Dimensions de l'échiquier
taille_case = 50
echiquier_size = 8
chemin_images = "chess_images"
canvas = ipycanvas.Canvas(width=taille_case * echiquier_size, height=taille_case * echiquier_size)
display(canvas)

# Initialisation de Stockfish
stockfish = Stockfish("/usr/games/stockfish")  
stockfish.set_skill_level(10)

# Initialisation de l'échiquier
echiquier = chess.Board()

# Fonction pour charger une partie PGN
def charger_partie_pgn(fichier_pgn):
    with open(fichier_pgn, "r") as pgn:
        partie = chess.pgn.read_game(pgn)
    return list(partie.mainline_moves())

# Fonction pour dessiner l'échiquier
def dessiner_echiquier():
    for y in range(echiquier_size):
        for x in range(echiquier_size):
            color = "white" if (x + y) % 2 == 0 else "grey"
            canvas.fill_style = color
            canvas.fill_rect(x * taille_case, y * taille_case, taille_case, taille_case)

# Fonction pour dessiner les pièces sur l'échiquier
def dessiner_piece_image(image_path, x, y):
    if os.path.exists(image_path):
        with open(image_path, "rb") as file:
            image = widgets.Image(value=file.read(), format='png')
        canvas.draw_image(image, x * taille_case, y * taille_case, taille_case, taille_case)

# Fonction pour mettre à jour l'échiquier
def mettre_a_jour_echiquier():
    canvas.clear()
    dessiner_echiquier()
    for case in chess.SQUARES:
        piece = echiquier.piece_at(case)
        if piece:
            x, y = chess.square_file(case), 7 - chess.square_rank(case)
            image_path = os.path.join(chemin_images, f"{'w' if piece.color else 'b'}{piece.symbol().upper()}.png")
            dessiner_piece_image(image_path, x, y)

# Fonction pour afficher une annotation sur le canvas
def afficher_annotation_image(annotation, coup):
    print(f"Annotation à afficher: {annotation}") 
    if annotation:
        # Exemple d'annotation, vous pouvez ajouter plus de conditions selon le cas
        image_path = ""
        if annotation == "!!":
            image_path = "annotations_images/brilliant.png"
        elif annotation == "!":
            image_path = "annotations_images/tres bon.png"
        elif annotation == "?":
            image_path = "annotations_images/interessant.png"
        elif annotation == "??":
            image_path = "annotations_images/gaffe.png"
        elif annotation == "?!?":
            image_path = "annotations_images/imprecision.png"

        print(f"Chemin de l'image: {image_path}")  

        # Vérifier si l'image existe
        if os.path.exists(image_path):
            print(f"Image trouvée: {image_path}")
            
            # Exemple pour afficher l'image sur une case fixe (par exemple D4)
            case = chess.parse_square("d4")  # Case D4
            x, y = chess.square_file(case), 7 - chess.square_rank(case)  # Coordonnées de la case
            
            # Position de l'annotation
            annotation_x = (x + 1) * taille_case - taille_case // 2  # Décalage à droite
            annotation_y = (y) * taille_case + 10  # Décalage vers le haut

            try:
                # Lire l'image en mode binaire
                with open(image_path, "rb") as img_file:
                    img_data = img_file.read()

                # Créer l'image pour l'afficher
                image_widget = widgets.Image(value=img_data, format='png')

                # Dessiner l'image sur le canvas
                canvas.draw_image(image_widget, annotation_x, annotation_y, taille_case, taille_case)

            except Exception as e:
                print(f"Erreur lors du chargement de l'image : {e}")
        else:
            print(f"Erreur: L'image d'annotation {image_path} n'a pas été trouvée.")
    else:
        print("Aucune annotation à afficher.")

# Test de l'affichage d'annotation
afficher_annotation_image("!!", None)  # Affiche une annotation pour le test


Canvas(height=400, width=400)

Annotation à afficher: !!
Chemin de l'image: annotations_images/brilliant.png
Erreur: L'image d'annotation annotations_images/brilliant.png n'a pas été trouvée.
